In [1]:
# python env: atom2_env_dev_v2

import json                                                                                                                             
import monty.json
import numpy as np
import pandas as pd
import traceback as tb
import sys
 
from pymatgen.core.structure import Structure
 
from atomate2.vasp.jobs.core import DielectricMaker
from atomate2.vasp.powerups import (
    update_user_incar_settings,
    update_user_potcar_functional,
    update_user_kpoints_settings
)
 
from fireworks import LaunchPad
from importlib import reload
 
from jobflow import Flow
from jobflow import run_locally
from jobflow.managers.fireworks import flow_to_workflow
 
from pathlib import Path
import pickle
from tqdm import tqdm

/home/vtrinquet/anaconda3/envs/atom2env_dev_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LaunchPad

In [3]:
v_iter = "v3"

#   Paths to the candidates datasets
candidate_df_path    = Path(f"df_selected_{v_iter}.json")
 
#   Loading the candidates datasets
if not candidate_df_path.exists():
    raise Exception(f"The file containing the structures of the candidates was not found at {candidate_df_path}.")
else:
    df_selected = pd.read_json(candidate_df_path)

#   Iterating over the candidates to add them in the Launchpad 
for mpid, d in tqdm(zip(df_selected.index.values, df_selected['structure'])):

    # Initializes the MP structure
    struc = Structure.from_dict(d)
    structure = struc.get_primitive_structure()
 
    # Create a Dfpt dielectric WF to compute the dielectric tensor
    dielec_flow = DielectricMaker().make(structure=structure)
 
    # Power ups
        # Increase the kpoints density to 1500 per reciprocal atom for balance accuracy/complexity
    dielec_flow = update_user_kpoints_settings( dielec_flow, {"grid_density": 1500})
 
    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    dielec_flow.update_metadata({"Batch": f"re2fractive_hg_{v_iter}", "mp-id": f"{mpid}"})
 
 
 
        # Specify:  the parallelization to optimize performance
        #           the electronic convergence criterion            (1E-5 eV otherwise by default),
        #           do not enforce a type of psp --> read in POTCAR (PBEsol otherwise by default)
        #           projection done in real space to improve efficiency (True otherwise)
    dielec_flow = update_user_incar_settings(   dielec_flow, {  "KPAR": 8,
                                                                "EDIFF": 1E-6,
                                                                "GGA": None,
                                                                "LREAL": 'False',
                                                                "ALGO": 'All',
                                                                "ISMEAR": 0,
                                                                "SIGMA": 0.03,
                                                                "LAECHG": 'False',
                                                                "LELF": 'False',
                                                                "LVTOT": 'False',
                                                                "LWAVE": 'False',
                                                                "PREC": 'Accurate',
                                                                "IBRION": -1,
                                                                "NSW": 0
                                                                })
 
        # Choose the type of PSP, here PBE_54
    dielec_flow = update_user_potcar_functional(dielec_flow, "PBE_64")
 
    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(dielec_flow)

    # submit the workflow to the FireWorks launchpad (requires a valid connection)
    lpad = LaunchPad.auto_load()
    lpad.add_wf(wf)
 
#   Final step: launch the calculations on a cluster using qlaunch 

0it [00:00, ?it/s]

2024-01-23 15:08:08,273 INFO Added a workflow. id_map: {-8: 8548}
2024-01-23 15:08:08,306 INFO Added a workflow. id_map: {-9: 8549}


/home/vtrinquet/Softwares_Packages/Github/atomate2/src/atomate2/vasp/sets/base.py:360: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended as it can significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(
2it [00:00, 16.19it/s]

2024-01-23 15:08:08,361 INFO Added a workflow. id_map: {-10: 8550}
2024-01-23 15:08:08,428 INFO Added a workflow. id_map: {-11: 8551}


4it [00:00, 16.29it/s]

...

196it [00:09, 19.93it/s]

2024-01-23 15:08:18,150 INFO Added a workflow. id_map: {-204: 8744}
2024-01-23 15:08:18,180 INFO Added a workflow. id_map: {-205: 8745}
2024-01-23 15:08:18,240 INFO Added a workflow. id_map: {-206: 8746}


199it [00:10, 20.49it/s]

2024-01-23 15:08:18,293 INFO Added a workflow. id_map: {-207: 8747}


200it [00:10, 19.78it/s]


In [2]:
#   Paths to the candidates datasets
candidate_df_path    = Path("df_redo_yb.json")
 
#   Loading the candidates datasets
if not candidate_df_path.exists():
    raise Exception(f"The file containing the structures of the candidates was not found at {candidate_df_path}.")
else:
    df_selected = pd.read_json(candidate_df_path)

#   Iterating over the candidates to add them in the Launchpad 
for mpid, d in tqdm(zip(df_selected.index.values, df_selected['structure'])):

    # Initializes the MP structure
    struc = Structure.from_dict(d)
    structure = struc.get_primitive_structure()
 
    # Create a Dfpt dielectric WF to compute the dielectric tensor
    dielec_flow = DielectricMaker().make(structure=structure)
 
    # Power ups
        # Increase the kpoints density to 1500 per reciprocal atom for balance accuracy/complexity
    dielec_flow = update_user_kpoints_settings( dielec_flow, {"grid_density": 1500})
 
    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    dielec_flow.update_metadata({"Batch": "re2fractive_hg_v3_redo_yb", "mp-id": f"{mpid}"})
 
 
 
        # Specify:  the parallelization to optimize performance
        #           the electronic convergence criterion            (1E-5 eV otherwise by default),
        #           do not enforce a type of psp --> read in POTCAR (PBEsol otherwise by default)
        #           projection done in real space to improve efficiency (True otherwise)
    dielec_flow = update_user_incar_settings(   dielec_flow, {  "KPAR": 8,
                                                                "EDIFF": 1E-6,
                                                                "GGA": None,
                                                                "LREAL": 'False',
                                                                "ALGO": 'All',
                                                                "ISMEAR": 0,
                                                                "SIGMA": 0.03,
                                                                "LAECHG": 'False',
                                                                "LELF": 'False',
                                                                "LVTOT": 'False',
                                                                "LWAVE": 'False',
                                                                "PREC": 'Accurate',
                                                                "IBRION": -1,
                                                                "NSW": 0
                                                                })
 
        # Choose the type of PSP, here PBE_54
    dielec_flow = update_user_potcar_functional(dielec_flow, "PBE_64")
 
    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(dielec_flow)

    # submit the workflow to the FireWorks launchpad (requires a valid connection)
    lpad = LaunchPad.auto_load()
    lpad.add_wf(wf)
 
#   Final step: launch the calculations on a cluster using qlaunch 

0it [00:00, ?it/s]

2024-01-23 14:48:47,090 INFO Added a workflow. id_map: {-1: 8541}
2024-01-23 14:48:47,132 INFO Added a workflow. id_map: {-2: 8542}
2024-01-23 14:48:47,167 INFO Added a workflow. id_map: {-3: 8543}


/home/vtrinquet/Softwares_Packages/Github/atomate2/src/atomate2/vasp/sets/base.py:360: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended as it can significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(
3it [00:00, 25.46it/s]

2024-01-23 14:48:47,203 INFO Added a workflow. id_map: {-4: 8544}
2024-01-23 14:48:47,236 INFO Added a workflow. id_map: {-5: 8545}
2024-01-23 14:48:47,269 INFO Added a workflow. id_map: {-6: 8546}


6it [00:00, 27.61it/s]

2024-01-23 14:48:47,303 INFO Added a workflow. id_map: {-7: 8547}


7it [00:00, 27.54it/s]
